<a href="https://colab.research.google.com/github/tig1795/word_embeddings_Lexika/blob/main/Fine_tuning_ohne_Label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports und Installationscheck

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch
torch.cuda.empty_cache()

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
!pip install datasets transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.17.0


# Upload der Daten

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': ['train.csv'], 'validation': 'test.csv'})

Using custom data configuration default-2a83e911cd55407c


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2a83e911cd55407c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# Vorbereiten des Datensatzes 

In [ ]:
dataset["train"][10]

{'text': 'Die Mildenburg ist eine um 1200 erbaute Höhenburg oberhalb der Kreisstadt Miltenberg im unterfränkischen Landkreis Miltenberg in Bayern.Der Name Mildenburg, ursprünglich Mildenberg oder Miltenberg, besteht aus den mittelhochdeutschen Wörtern milte im Sinne von gütig oder freigebig und bërc für Berg oder Burg. Sie sollten die Eigenschaften der Burg und ihrer Herren charakterisieren. Der Burgname ging auf den Ort Miltenberg über.\nTeilweise wurde ein lateinischer Namensteil verwendet: Piemontanus (gütiger Berg).Die Burg wurde gegen Ende des 12. Jahrhunderts vom Mainzer Erzbischof als östliche Grenzsicherung seines Einflussbereiches auf dem Schloßberg, einem nördlichen Vorsprung des Greinbergs errichtet und 1226 erstmals urkundlich erwähnt. Der aus Buckelquadern erbaute, 27 Meter hohe Bergfried ist der älteste Teil der Burganlage. Er ist an der meistgefährdeten Stelle, dem ansteigenden Hang gegen Süden, angelegt.\nDas heutige Wohngebäude mit hohem Dach und Treppengiebel wurde vo

In [ ]:
dataset["validation"][10]

{'text': 'The Slap – Nur eine Ohrfeige (Originaltitel: The Slap) ist eine achtteilige, australische Fernseh-Miniserie. Sie beruht auf dem mehrfach ausgezeichneten Roman Nur eine Ohrfeige des australischen Autors Christos Tsiolkas. Ausgangspunkt der Handlung ist, dass ein Mann ein Kind ohrfeigt, mit dem er nicht verwandt ist. Die Serie wurde vom australischen Sender ABC1 zwischen dem 6. Oktober und dem 24. November 2011 erstmals ausgestrahlt. Die Erstausstrahlung der deutschen Synchronfassung war vom 5. September bis 26. September 2013 auf Arte zu sehen. Kritiker nahmen The Slap – Nur eine Ohrfeige überwiegend wohlwollend auf. Die Serie erhielt mehrere australische Fernsehpreise und war international unter anderem für einen Emmy nominiert.Die Hauptfiguren sind zu Beginn der Handlung durch Freundschaft oder Familie miteinander verbunden. Die freundschaftlichen Bindungen ändern sich teilweise im Laufe der Serie. Hector ist ein Australier griechischer Abstammung. Er ist vierzig Jahre alt u

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

# Causal Language modeling

In [ ]:
model_checkpoint = "redewiedergabe/bert-base-historical-german-rw-cased"

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=None, truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets["train"][1]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [ ]:
tokenized_datasets["validation"][1]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'HttpClient um die HTTP - Verbindungen aufzubauen. Cactus sieht sich als Framework für integrative Unit - Tests [UNK] d. h. Unit - Tests, die die Interaktionen mit dem Container testen. Cactus ist aber auch für logische Unit - Tests ( z. B. Applikationslogiktest ) und funktionelle Unit - Tests ( z. B. HTTP Request - / Response - Test ) einsetzbar. Cactus wird seit dem 5. August 2011 nicht mehr weiterentwickelt. Cactus ermöglicht es Uni'

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at redewiedergabe/bert-base-historical-german-rw-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"bert-base-historical-german-rw-cased-finetuned-wikitext",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
    #push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 19093
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7161


Epoch,Training Loss,Validation Loss
1,0.006700,0.000371
2,0.002100,0.000339
3,0.000900,0.000081


Saving model checkpoint to bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-500
Configuration saved in bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-500/config.json
Model weights saved in bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-500/pytorch_model.bin
Saving model checkpoint to bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-1000
Configuration saved in bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-1000/config.json
Model weights saved in bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-1500
Configuration saved in bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-1500/config.json
Model weights saved in bert-base-historical-german-rw-cased-finetuned-wikitext/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to bert-base-historica

TrainOutput(global_step=7161, training_loss=0.09443921713069271, metrics={'train_runtime': 4120.44, 'train_samples_per_second': 13.901, 'train_steps_per_second': 1.738, 'total_flos': 3769004155170816.0, 'train_loss': 0.09443921713069271, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 4793
  Batch size = 8


Perplexity: 1.00


In [ ]:
import os

os.makedirs("finetunedmodell")

In [ ]:
model.save_pretrained("finetunedmodell/old-bert-finetuned-wiki")

Configuration saved in finetunedmodell/old-bert-finetuned-wiki/config.json
Model weights saved in finetunedmodell/old-bert-finetuned-wiki/pytorch_model.bin


In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("finetunedmodell/old-bert-finetuned-wiki")

loading configuration file finetunedmodell/old-bert-finetuned-wiki/config.json
Model config BertConfig {
  "_name_or_path": "finetunedmodell/old-bert-finetuned-wiki",
  "architectures": [
    "BertLMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file finetunedmodell/old-bert-finetuned-wiki/pytorch_model.bin
Some weights of the model checkpoint at finetunedmodell/old-bert-finetuned-wiki were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.pre

# Anmwendung des fine getunten Models

In [ ]:
text = "bank"
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'b', '##ank', '[SEP]']


In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [ ]:
indexed_tokens

[3, 40, 937, 4]

In [ ]:
segments_ids = [1] * len(tokenized_text)

In [ ]:
from transformers import AutoModel

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = AutoModel.from_pretrained("finetunedmodell/old-bert-finetuned-wiki")

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

loading configuration file finetunedmodell/old-bert-finetuned-wiki/config.json
Model config BertConfig {
  "_name_or_path": "finetunedmodell/old-bert-finetuned-wiki",
  "architectures": [
    "BertLMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file finetunedmodell/old-bert-finetuned-wiki/pytorch_model.bin
Some weights of the model checkpoint at finetunedmodell/old-bert-finetuned-wiki were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
tokens_tensor

tensor([[  3,  40, 937,   4]])

In [ ]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
encoded_layers

'last_hidden_state'

In [ ]:
from transformers import BertTokenizer, BertModel

import torch

#tokenizer = BertTokenizer.from_pretrained("finetunedmodell/old-bert-finetuned-wiki")

model = BertModel.from_pretrained("finetunedmodell/old-bert-finetuned-wiki")

word = "bank"

inputs = tokenizer(word, return_tensors="pt")

outputs = model(**inputs)

word_vect = outputs.pooler_output.detach().numpy()

loading configuration file finetunedmodell/old-bert-finetuned-wiki/config.json
Model config BertConfig {
  "_name_or_path": "redewiedergabe/bert-base-historical-german-rw-cased",
  "architectures": [
    "BertLMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file finetunedmodell/old-bert-finetuned-wiki/pytorch_model.bin
Some weights of the model checkpoint at finetunedmodell/old-bert-finetuned-wiki were not used when initializing BertModel: ['cls.predictions.transform.den

In [ ]:
w1 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()
w2 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()

In [ ]:
w2.shape

(1, 768)

In [ ]:
w1.shape

(1, 768)

In [ ]:
from numpy import dot
from numpy.linalg import norm

a = w1
b = w2.T

cos_sim = dot(a, b)/(norm(a)*norm(b))

In [ ]:
cos_sim[0][0]

0.99999994

In [ ]:
import operator
def foo(w, l):
  w1 = model(**tokenizer(w, return_tensors="pt")).pooler_output.detach().numpy()

  output = {}

  for tw in l:
    w2 = model(**tokenizer(tw, return_tensors="pt")).pooler_output.detach().numpy()
    w2 = w2.T
    cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))
    output[tw] = cos_sim[0][0] # schlauer mit np.squeeze

  return sorted(output.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
bat = "Batterie" 
batt = ["Brigade", "Kompanie", "Division", "Infanterie", "Regiment", "Artillerie"]

batterie = foo(bat, batt)

In [ ]:
batterie

[('Infanterie', 0.90030676),
 ('Brigade', 0.89334965),
 ('Artillerie', 0.89286476),
 ('Division', 0.8850176),
 ('Kompanie', 0.8829111),
 ('Regiment', 0.8483296)]

In [ ]:
batt2 = ["Diesel", "Tank", "Speicher", "Elektronik", "Prototyp", "Laden", "Strom"]

batterie2 = foo(bat, batt2)

In [ ]:
batterie2

[('Speicher', 0.8486875),
 ('Strom', 0.8434096),
 ('Tank', 0.8119138),
 ('Elektronik', 0.8112367),
 ('Diesel', 0.8051002),
 ('Laden', 0.7909585),
 ('Prototyp', 0.7885536)]

In [ ]:
batt3 = ["Diesel", "Tank", "Speicher", "Elektronik", "Brigade", "Kompanie", "Division", "Infanterie", "Regiment", "Artillerie", "Prototyp", "Laden", "Strom"]

batterie3 = foo(bat, batt3)

In [ ]:
batterie3

[('Infanterie', 0.90030676),
 ('Brigade', 0.89334965),
 ('Artillerie', 0.89286476),
 ('Division', 0.8850176),
 ('Kompanie', 0.8829111),
 ('Speicher', 0.8486875),
 ('Regiment', 0.8483296),
 ('Strom', 0.8434096),
 ('Tank', 0.8119138),
 ('Elektronik', 0.8112367),
 ('Diesel', 0.8051002),
 ('Laden', 0.7909585),
 ('Prototyp', 0.7885536)]

Immer noch alte Bedeutung dominant, allerdings sind die Werte näher an einander gerückt.